In [ ]:
import os
import numpy as np
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import math
from scipy import signal
from scipy.signal import savgol_filter
from scipy import fftpack
from scipy.fftpack import fft,ifft
from scipy.signal import freqs
import pandas as pd

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Read video with OpenCV.
cap=cv2.VideoCapture('/home/johnson/Desktop/UPDRS/UPDRS_video/Tim_Tremor/t014_crop_256_10s.mp4')
# Read sensor data
tsv_data = pd.read_csv('/home/johnson/tremor_dataset/T014_Right/Rest/kinect_accelerometer.tsv', sep='\t') 

## Get video info
RES = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fps = round(cap.get(cv2.CAP_PROP_FPS))

#create video writer to write detected_video
#output_video = "../UPDRS_result/t014_tremor.mp4"
#fourcc = cv2.VideoWriter_fourcc(*'MP4V')
#out = cv2.VideoWriter(output_video, fourcc, 30, RES)

In [ ]:
#single hand (grasp)
hands = mp_hands.Hands(static_image_mode=True,max_num_hands=2,min_detection_confidence=0.7)

record = []
frame_count = 0
img_list = []
lost_frame = []

while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        frame = frame[:,:]
        frame_count += 1

        # Convert the BGR image to RGB, flip the image around y-axis for correct 
        # handedness output and process it with MediaPipe Hands.
        results = hands.process(cv2.flip(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 1))
        annotated_image = cv2.flip(frame.copy(), 1)

        if results.multi_hand_landmarks != None:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                #middle_y = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x
                record.append(hand_landmarks.landmark)

        else:
            lost_frame.append(frame_count)

        #after processing the hand, flip back the image
        annotated_image = cv2.flip(annotated_image, 1)
        img_list.append(annotated_image)
        #out.write(annotated_image)

    else:
        break

cap.release()
#out.release()
#plt.plot(record)

In [ ]:
sensor_x = tsv_data["-326"][0:10000] - np.mean(tsv_data["-326"][0:10000])
sensor_x = np.array(sensor_x)
plt.plot(sensor_x)

In [ ]:
copy = img_list[0].copy()
plt.imshow(cv2.cvtColor(copy, cv2.COLOR_BGR2RGB))

In [ ]:
for index in range(21):
    fig, (ax1, ax2) = plt.subplots(2)
    x = [i[index].x for i in record]
    y = [i[index].y for i in record]
    ax1.plot(x) ; ax1.title.set_text(f'{index} (x-axis)')
    ax2.plot(y) ; ax2.title.set_text(f'{index} (y-axis)')
    fig.tight_layout()
#a = [i[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y for i in record]
#plt.plot(a)

In [ ]:
def draw_frequency(frequency,amplitude):
    freqs = []
    amps = []
    for freq,amp in zip(frequency,amplitude):
        if 0<=freq<=14:
            freqs.append(freq)
            amps.append(abs(amp))
    print(frequency[np.argmax(np.abs(amplitude))])
    plt.plot(freqs,amps)

def draw_video_frequency(frequency_x,amplitude_x,frequency_y,amplitude_y,index):
    freqs_x = []
    amps_x = []
    freqs_y = []
    amps_y = []

    for freq,amp in zip(frequency_x,amplitude_x):
        if 0<=freq<=14:
            freqs_x.append(freq)
            amps_x.append(abs(amp))
    for freq,amp in zip(frequency_y,amplitude_y):
        if 0<=freq<=14:
            freqs_y.append(freq)
            amps_y.append(abs(amp))

    
    fig, (ax1, ax2) = plt.subplots(2)
    ax1.plot(freqs_x,amps_x) ; ax1.title.set_text(str(index) + " (x-axis)")
    ax2.plot(freqs_y,amps_y) ; ax2.title.set_text(str(index) + " (y-axis)")
    fig.tight_layout()


def find_frequency(frequency,amplitude):
    amp_sort = sorted(amplitude,reverse=True)
    maximum = amp_sort[0]
    domain_frequency = frequency[np.argmax(amplitude)]
    if 3 <= domain_frequency <= 8:
        return domain_frequency

    for amp in amp_sort:
        if(maximum * 0.4 <= amp):
            index = np.where(amplitude == amp)
            freq = frequency[index]
            if(3<= freq <= 8):
                return freq[0]

        else:
            return domain_frequency


sensor_X = fftpack.fft(sensor_x)
sensor_freqs = fftpack.fftfreq(len(sensor_x)) * 1000
draw_frequency(sensor_freqs,sensor_X)

for index in range(21):
    x = [i[index].x for i in record]
    x = np.array(x)
    x = x - np.mean(x)

    y = [i[index].y for i in record]
    y = np.array(y)
    y = y - np.mean(y)

    X = fftpack.fft(x)
    x_freqs = fftpack.fftfreq(len(x)) * 30 #(len(a)/len(img_list))*30
    freq_x = find_frequency(x_freqs[:int(x_freqs.size/2)],np.abs(X[:int(X.size/2)]))

    Y = fftpack.fft(y)
    y_freqs = fftpack.fftfreq(len(y)) * 30 #(len(a)/len(img_list))*30
    freq_y = find_frequency(y_freqs[:int(y_freqs.size/2)],np.abs(Y[:int(Y.size/2)]))
    #print(f'{index:3}. {x_freqs[np.argmax(np.abs(X))]:.4f}, {y_freqs[np.argmax(np.abs(Y))]:.4f}')
    print(f'{index:3}. {freq_x:.4f}, {freq_y:.4f}')
    
    draw_video_frequency(x_freqs,np.abs(X),y_freqs,np.abs(Y),index)
    


In [ ]:
mid = (min(a) + max(a))/2
pro = (max(a) - mid)

peaks, _ = signal.find_peaks(a,height=mid,prominence=pro)
valley, _ = signal.find_peaks(np.array(a)*-1,height=mid*-1)

plt.plot(a)
plt.plot(peaks,np.array(a)[peaks],"x")
plt.plot(valley,np.array(a)[valley],"o")